# EMT::Ph1::Switch Test

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import dpsimpy

#%matplotlib widget

## VS_SW_R_L - Circuit

In [ ]:
time_step = 0.00001
final_time = 0.1
sim_name = 'EMT_VS_SW_RL1'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)

gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode('n1')
n2 = dpsimpy.emt.SimNode('n2')
n3 = dpsimpy.emt.SimNode('n3')

resistance = 5
inductance = 0.02
open_resistance = 1e6
closed_resistance = 0.001

vs = dpsimpy.emt.ph1.VoltageSource('vs')
vs.set_parameters(complex(10, 0), 50)
res = dpsimpy.emt.ph1.Resistor('Resistor')
res.set_parameters(resistance)
ind = dpsimpy.emt.ph1.Inductor('Inductance')
ind.set_parameters(inductance)
sw = dpsimpy.emt.ph1.Switch('Switch')
sw.set_parameters(open_resistance, closed_resistance, False)

vs.connect([gnd, n1])
sw.connect([n1, n2])
res.connect([n2, n3])
ind.connect([n3, gnd])

sys = dpsimpy.SystemTopology(50, [n1, n2, n3], [vs, sw, res, ind])

logger = dpsimpy.Logger(sim_name)
logger.log_attribute('v2', 'v', n2)
logger.log_attribute('v3', 'v', n3)
logger.log_attribute('i_res', 'i_intf', res)

sim = dpsimpy.Simulation(sim_name)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.add_logger(logger)

event_sw = dpsimpy.event.SwitchEvent(0.04, sw, True)
sim.add_event(event_sw)

sim.run()

In [ ]:
work_dir = 'logs/EMT_VS_SW_RL1/'
log_name = 'EMT_VS_SW_RL1'
print(work_dir + log_name + '.csv')
EMTseries = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

In [ ]:
plt.figure()
plt.plot(EMTseries['v2'].time, EMTseries['v3'].values - EMTseries['v2'].values, label='V_res')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(EMTseries['i_res'].time, EMTseries['i_res'].values, label='I')
plt.legend()
plt.show()